# Ingest Summoner Champions API to Bronze
This notebook loads the summoner champions data from the League of Legends API. Cf. detail on the API by Riot Games on: https://developer.riotgames.com/

It requires the list of summoners for which we want to get data as input.

The list of summoners from which you want to get data is a csv file was created for that and manually ingested in the Files section of the Bronze Lakehouse. The csv file contains summoner name (which can change) ,the Player Universally Unique ID (puuid, which cannot change) and the Summoner ID (which is needed for some API calls). The puuid was fetched using the summoner API : https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"summoner-name"

For each summoner, we extract all information about the champions he plays and store that information in a json file in the Bronze Lakehouse.

### 1. General configuration

In [8]:
# Import libraries
from trident_token_library_wrapper import PyTridentTokenLibrary as tl
import requests as r
import json
from datetime import datetime
import os

StatementMeta(, 76de1fc1-68a6-4a56-bb0c-983a72ecb087, 10, Finished, Available)

In [3]:
# Load secrets from Azure Key Vault
key_vault_name = 'testasa-akv-dev-001'
key_name = 'lol-api-key'
access_token = mssparkutils.credentials.getToken("keyvault")
api_key = tl.get_secret_with_token(f"https://{key_vault_name}.vault.azure.net/", key_name, access_token)

# Define API endpoint and parameters
base_url = 'https://euw1.api.riotgames.com/lol/'

StatementMeta(, 76de1fc1-68a6-4a56-bb0c-983a72ecb087, 5, Finished, Available)

### 2. Load Summoner Champions data

In [10]:
# Read Summoners.csv into a DataFrame
print("Reading summoners csv file into a DataFrame")
df_summoners = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ";").csv("Files/raw_data/Summoners.csv")
print("Finished reading summoners csv file into a DataFrame, found", df_summoners.count(), "summoners")

# Make a list out of the PUUIDs to iterate on PUUIDs
puuid_list = df_summoners.select("PUUID").collect()

StatementMeta(, 76de1fc1-68a6-4a56-bb0c-983a72ecb087, 12, Finished, Available)

Reading summoners csv file into a DataFrame
Finished reading summoners csv file into a DataFrame, found 10 summoners


In [ ]:
# Make API calls to get champions of each summoner and write json file to Bronze Lakehouse

print("Starting API calls to get summoner champions")

current_date = datetime.now().strftime("%Y/%m/%d")

for puuid_row in puuid_list:
    print(f"Getting data for puuid: {puuid_row[0]}")
    puuid = puuid_row[0]

    # Make API call to fetch summoner champions detail 
    params = {'api_key': api_key}
    response = r.get(f"{base_url}champion-mastery/v4/champion-masteries/by-puuid/{puuid}", params=params)

    # Write data back to lakehouse keeping original json format
    if response.status_code == 200:
        outputFilePath = f"//lakehouse/default/Files/raw_data/summoner_champions/{current_date}/{puuid}.json"
 
        if not os.path.exists(os.path.dirname(outputFilePath)):    
            print("creating folder: " + os.path.dirname(outputFilePath))    
            os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
 
        with open(outputFilePath, 'wb') as f:
            f.write(response.content)

    else:
        print(f"Failed to fetch data for puuid: {puuid}")